# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

##### Import PySpark module and create pyspark session.

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer

spark = SparkSession.builder.appName('lr_project').getOrCreate()

##### Import given dataset and explore columns and datatypes.

In [2]:
data = spark.read.csv("cruise_ship_info.csv", 
                      inferSchema=True, 
                      header=True)
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [3]:
data.count()

158

In [4]:
data.head(5)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)]

##### There is an insight that the Cruise line column will be a very helpful categorical variable for our regression model. Thus we are going to convert it to integer.

In [5]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [6]:
# Create new columnd with cruise line id
labeles_cruise_lines = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_id")
labeles_cruise_lines_model = labeles_cruise_lines.fit(data)
data = labeles_cruise_lines_model.transform(data)

# Delete original column
data = data.drop("Cruise_line")

##### Setting up DataFrame for ML

In [7]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['Ship_name',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_id']

In [9]:
assembler = VectorAssembler(
    inputCols = 
    ['Age',
     'Tonnage',
     'passengers',
     'length',
     'cabins',
     'passenger_density',
     'Cruise_line_id'],
    outputCol='features'
                        )

In [10]:
output = assembler.transform(data)

In [11]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [12]:
# Selecting the dependant variable and features in final dataframe

final_data = output.select("features", "crew")
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [13]:
# Divinding final data frame into train and tesst data

train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [14]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               111|
|   mean| 7.668468468468478|
| stddev|3.3921034499121343|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [15]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                47|
|   mean| 8.091063829787233|
| stddev|3.7748787680173903|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



##### Ar. mean and stddev is pretty close hence distribution of the train and test should be considered as close to the similar.
##### Next, proceeding to Linear Regression model

In [16]:
# Create a Linear Regression Model object

lr = LinearRegression(labelCol="crew")

# Fit the model to the data and call this model lrModel

lrModel = lr.fit(train_data)

In [17]:
# Print the coefficients and intercept for linear regression

print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.015610777832109334,0.020192398805657163,-0.16168446537153108,0.42318322397418145,0.7694120202431516,-0.010656514325009725,0.04219148022975241] Intercept: -0.5183278760065753


In [18]:
# Test LR model on the test dataframe

test_results = lrModel.evaluate(test_data)

In [19]:
test_results.rootMeanSquaredError

1.3333061233232182

In [20]:
test_results.r2

0.8725341764383886

In [21]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-0.10118008992108951|
|   0.275114400683675|
| -1.3755961048173706|
| -0.7930734610886745|
| -0.9951246276240209|
|  0.6749740307027263|
|  0.9807399667669294|
| -0.3827331760635353|
|  1.1349765797610463|
| -0.3914655503129154|
| -0.2473724090078271|
|  0.9774152630254971|
|-0.16944113769916136|
|   7.247584562647138|
| -1.0355676761895225|
|-0.09838726547247756|
|  0.2622697653937145|
|  0.9057147118219504|
|  0.7821417764004384|
| -0.5291630709892869|
+--------------------+
only showing top 20 rows



##### Forecasting for unlabeled test data using our regression model

In [22]:
unlabeled_data = test_data.select("features")

In [23]:
predictions = lrModel.transform(unlabeled_data)

In [24]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,122.0,28.5,1...|  6.80118008992109|
|[5.0,133.5,39.59,...|12.854885599316326|
|[5.0,160.0,36.34,...| 14.97559610481737|
|[6.0,30.276999999...| 4.343073461088674|
|[6.0,90.0,20.0,9....|  9.99512462762402|
|[6.0,113.0,37.82,...|11.325025969297274|
|[8.0,91.0,22.44,9...| 10.01926003323307|
|[9.0,105.0,27.2,8...|11.062733176063535|
|[9.0,113.0,26.74,...|11.245023420238955|
|[10.0,81.76899999...| 8.811465550312915|
|[10.0,91.62700000...| 9.247372409007827|
|[10.0,151.4,26.2,...|11.552584736974502|
|[11.0,86.0,21.24,...| 9.469441137699162|
|[11.0,110.0,29.74...|11.852415437352864|
|[11.0,138.0,31.14...|12.885567676189522|
|[12.0,25.0,3.88,5...|2.9683872654724777|
|[12.0,42.0,14.8,7...| 6.537730234606285|
|[12.0,77.104,20.0...|  8.68428528817805|
|[12.0,91.0,20.32,...| 9.207858223599562|
|[13.0,61.0,13.8,7...| 6.529163070989287|
+--------------------+------------

###### Checking model for the reality

In [25]:
from pyspark.sql.functions import corr

In [26]:
data.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [27]:
data.select(corr('crew', 'cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

